In [1]:
import sys
sys.path.append('..')
from itertools import permutations
from modified_sage_funcs import *
from imp_funcs import *

n = 5
M = matroids.CompleteGraphic(5)
r = M.rank()
S = (frozenset({0, 1, 2, 3, 4}), frozenset({2, 3, 4, 5, 6, 7}))
k = len(S)
P = PolynomialRing(QQ, 'x', k)

In [2]:


def partitions(basis, weak_comp):
    """
    partitions a basis into weak components
    """
    parts = set()
    for perm in permutations(basis):
        partition = []
        start = 0
        for size in weak_comp:
            partition.append(frozenset(perm[start:start + size]))
            start += size
        parts.add(tuple(partition))
    return tuple(parts)

def base_partitions(bases, indexed_compositions):
    """
    partitions each basis into weak components
    """
    bases_partitions = {}
    for basis in bases:
        base_partitions = {}
        for i in indexed_compositions:
            base_partitions[i] = partitions(tuple(basis), indexed_compositions[i])
        bases_partitions[basis] = base_partitions
    return bases_partitions


In [3]:
def composition_weight(iwc, set_system, bp):
    """
    computes the weight of each composition
    """
    weights = {}
    for i in iwc:
        weight = 0
        for basis in bp:
            for basis_partition in bp[basis][i]:
                flag = True
                for x, y, z in zip(iwc[i], basis_partition, set_system):
                    if len(y.intersection(z)) == x:
                        continue
                    else:
                        flag = False
                        break
                if flag:
                    weight += 1
                    break
        weights[i] = weight
    return weights

In [4]:
def weighted_poly(M, set_system):
    d, n = M.full_rank(), len(M.groundset())
    k = len(set_system)
    P = PolynomialRing(QQ, 'x', k)
    bases = sorted([frozenset(sorted(X)) for X in bases_iterator(M)])
    indexed_compositions = {i : x for i, x in enumerate(weak_compositions(d, k))}
    bp = base_partitions(bases, indexed_compositions)
    weights = composition_weight(indexed_compositions, set_system, bp)
    def monomial(wc):
        return prod([P.gen(i)**wc[i] for i in range(len(wc))])
    def weighted_sum(weights):
        return sum([weights[i]*monomial(indexed_compositions[i]) for i in weights])
    return weighted_sum(weights)

In [5]:
def decompose(S):
    return tuple(frozenset([x]) for sets in S for x in sets)

indexed_compositions = {i : x for i, x in enumerate(weak_compositions(r, k))}
indexed_compositions

{0: [1, 3], 1: [2, 2], 2: [3, 1], 3: [4, 0], 4: [0, 4]}

In [6]:
Sp = weighted_poly(M, S)
dSp = weighted_poly(M, decompose(S))

print(Sp)
print(dSp)

3*x0^4 + 18*x0^3*x1 + 36*x0^2*x1^2 + 34*x0*x1^3 + 11*x1^4
x0*x1*x2*x3 + x0*x2*x3*x4 + x1*x2*x3*x4 + x0*x1*x3*x5 + x0*x3*x4*x5 + x1*x3*x4*x5 + x0*x1*x2*x6 + x0*x2*x4*x6 + x1*x2*x4*x6 + x0*x1*x5*x6 + x0*x4*x5*x6 + x1*x4*x5*x6 + x0*x2*x3*x7 + x1*x2*x3*x7 + x0*x3*x5*x7 + x1*x3*x5*x7 + x0*x2*x6*x7 + x1*x2*x6*x7 + x0*x5*x6*x7 + x1*x5*x6*x7 + x0*x1*x3*x8 + x1*x2*x3*x8 + x0*x3*x4*x8 + x1*x3*x4*x8 + x2*x3*x4*x8 + x1*x3*x5*x8 + x3*x4*x5*x8 + x0*x1*x6*x8 + x1*x2*x6*x8 + x0*x4*x6*x8 + x1*x4*x6*x8 + x2*x4*x6*x8 + x1*x5*x6*x8 + x4*x5*x6*x8 + x0*x3*x7*x8 + x1*x3*x7*x8 + x2*x3*x7*x8 + x3*x5*x7*x8 + x0*x6*x7*x8 + x1*x6*x7*x8 + x2*x6*x7*x8 + x5*x6*x7*x8 + x0*x1*x2*x9 + x1*x2*x3*x9 + x0*x2*x4*x9 + x1*x2*x4*x9 + x2*x3*x4*x9 + x0*x1*x5*x9 + x1*x3*x5*x9 + x0*x4*x5*x9 + x1*x4*x5*x9 + x3*x4*x5*x9 + x1*x2*x6*x9 + x2*x4*x6*x9 + x1*x5*x6*x9 + x4*x5*x6*x9 + x0*x2*x7*x9 + x1*x2*x7*x9 + x2*x3*x7*x9 + x0*x5*x7*x9 + x1*x5*x7*x9 + x3*x5*x7*x9 + x2*x6*x7*x9 + x5*x6*x7*x9 + x0*x1*x8*x9 + x1*x2*x8*x9 + x1*x3*x8*x9 + x0*x

In [7]:
def specialize(f, S1, S2):
    P1 = PolynomialRing(QQ, 'x', len(S1))
    P2 = PolynomialRing(QQ, 'x', len(S2))
    k = 0
    for i, x in enumerate(S1):
        subs_dict = {P2.gen(k + j): P1.gen(i) for j in range(len(x))}
        f = f.subs(subs_dict)
        k += len(x)
    return f

print(specialize(dSp, S, decompose(S)))

3*x0^4 + 23*x0^3*x1 + 55*x0^2*x1^2 + 46*x0*x1^3 + 11*x1^4


In [8]:
def coefficients_dict(f):
    return {f.exponents()[i]: f.coefficients()[i] for i in range(len(f.coefficients()))}

coefficients_dict(specialize(dSp, S, decompose(S)))

{(4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0): 3,
 (3, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0): 23,
 (2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0): 55,
 (1, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0): 46,
 (0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0): 11}

In [9]:
print(Sp)
coefficients_dict(Sp)

3*x0^4 + 18*x0^3*x1 + 36*x0^2*x1^2 + 34*x0*x1^3 + 11*x1^4


{(4, 0): 3, (3, 1): 18, (2, 2): 36, (1, 3): 34, (0, 4): 11}

In [10]:
def normalize(f):
    return sum(f.monomials()[i] * f.coefficients()[i]/prod([factorial(x) for x in f.exponents()[i]]) for i in range(len(f.coefficients())))

nSp = normalize(Sp)
nSp

1/8*x0^4 + 3*x0^3*x1 + 9*x0^2*x1^2 + 17/3*x0*x1^3 + 11/24*x1^4

In [11]:
nSp.is_lorentzian()

True